In [1]:
##Liberias

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import glob
from datetime import datetime

## 1. Datos

In [2]:
#Normas

DF_normas = pd.read_csv("normas_binary.csv",index_col="Número de resolución")

#Tribunal
DF_tribunal = pd.read_csv("tribunal_binary.csv", index_col="Número de resolución")

#Empresas
DF_empresa = pd.read_csv("empresa_binary.csv", index_col="Número de resolución")

#TF-IDF todas
TF_Todas = np.loadtxt("TF-IDF_Vectorization_Todas.csv", delimiter=",")

#TF-IDF criterios
TF_Criterios = np.loadtxt("TF-IDF_Vectorization_Criterios.csv", delimiter=",")

#Word Emb criterios
WEmb_Criterios = np.loadtxt("Criterios_emb.csv", delimiter=",")


In [3]:
y = DF_normas.iloc[:, -1].values

In [4]:
normas_numpy = DF_normas.values[:, :-1]
tribunal_numpy = DF_tribunal.values[:, :-1]
empresa_numpy = DF_empresa.values[:, :-1]

## 2. Clasificación

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [15]:
def train_test_80_20(X, Y):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)
    return X_train, X_test, Y_train, Y_test

In [16]:
def test_evaluation(model, X_test, y_test):
    from sklearn.metrics import classification_report
    
    # evaluación en test del modelo
    y_test_pred = model.predict(X_test)
    print(classification_report(y_test, y_test_pred))

### 2.1. SVM

In [17]:
# 5-fold cross-validation
kf = KFold(n_splits=5,random_state=0)

In [9]:
now = datetime.now().time()
print("now =", now)

now = 18:53:59.043833


In [10]:
# optimización de hiperparámetros utilizando cross-validation

# MODELO: svm
# VARIABLES DE ENTRADA: criterios aplicables(TF-IDF)
# HYPERPARÁMETROS ÓPTIMOS: C=10, ..

pipeline = Pipeline([
    ("clf", SVC(kernel='rbf', C=1))
])

X = np.concatenate([TF_Todas], axis=1)
X_train, X_test, Y_train, Y_test = train_test_80_20(X, y)

scores = cross_val_score(pipeline, X_train, Y_train, 
                         cv=kf,
                         scoring="roc_auc"
                        )

print(f"{scores.mean():.3f} +- {scores.std():.3f}")

0.783 +- 0.078


In [11]:
pipeline.fit(X_train,Y_train)

Pipeline(steps=[('clf', SVC(C=1))])

In [12]:
test_evaluation(pipeline, X_test, Y_test)

              precision    recall  f1-score   support

           0       0.94      0.48      0.64        31
           1       0.61      0.96      0.75        26

    accuracy                           0.70        57
   macro avg       0.77      0.72      0.69        57
weighted avg       0.79      0.70      0.69        57



In [13]:
now = datetime.now().time()
print("now =", now)

now = 18:53:59.134054
